Imports

In [1]:
from Dataset import processData
from Model import train, KGLinkPredictor
import torch

Process Data

In [2]:
data, ptrain_loader, pval_loader, ftrain_loader, fval_loader, ftest_loader = processData(256,2048) # type: ignore

Loading data from path: /opt/scratch/labs/wuc/Drug-Repurposing/data/kg.csv
Processing node data...
Processing edge data...
Setting up pretraining data...
Setting up finetuning data...
Data processing complete.


Declare and Train Model

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = KGLinkPredictor(256,64,data).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
data = data.to(device)

print('Pretraining...')
train(ptrain_loader,pval_loader, model, optimizer, device, 1) # Takes about 30 mins

model.Decoder.reset_parameters()

print('Fine-tuning...')
train(ftrain_loader,fval_loader, model, optimizer, device, 50)

Pretraining...


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 | Average Training Loss: 0.14158970243103916 Average Validation Loss: 0.09911965072102896
Fine-tuning...


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1 | Average Training Loss: 0.8771406394593856 Average Validation Loss: 0.2618066022793452
Epoch 2 | Average Training Loss: 0.20643489764017217 Average Validation Loss: 0.2327298472325007
Epoch 3 | Average Training Loss: 0.1772311005522223 Average Validation Loss: 0.21591920653978983
Epoch 4 | Average Training Loss: 0.1720651125206667 Average Validation Loss: 0.1823930392662684
Epoch 5 | Average Training Loss: 0.1596242724096074 Average Validation Loss: 0.17535688479741415
Epoch 6 | Average Training Loss: 0.15083062561119304 Average Validation Loss: 0.18323520322640738
Epoch 7 | Average Training Loss: 0.1399695974062471 Average Validation Loss: 0.14557861785093942
Epoch 8 | Average Training Loss: 0.1377599874840063 Average Validation Loss: 0.1500058670838674
Epoch 9 | Average Training Loss: 0.12417588821228813 Average Validation Loss: 0.15469802419344583
Epoch 10 | Average Training Loss: 0.12841125752995997 Average Validation Loss: 0.16495227813720703
Epoch 11 | Average Training L

KeyboardInterrupt: 